In [3]:
import pandas as pd
import numpy as np
# import seaborn as sns
# import matplotlib.pyplot as plt
import os
import feather
# plt.style.use('seaborn-colorblind')
# %matplotlib inline
from feature_cleaning import outlier as ot

tree model 对于极值具有很强的鲁棒性，但是对于lr或者nn来说，异常值需要进行一定的处理，具体的我要去kaggle上看看tabular数据的nn

## 数据读取

In [5]:
X=pd.read_csv('X.csv')
X_test=pd.read_csv('X_test.csv')
cat=pd.read_csv('cat.csv')
cat=cat.cat.values.tolist()

C:\Users\ASUS\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (405,408,409,416,420,421,422,423,424,426,427,428,429,430,431,432,433) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\ASUS\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (404,407,408,415,419,420,421,422,423,425,426,427,428,429,430,431,432) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [7]:
X

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo,had_id
0,2987000,0,86400,68.50,W,13926,NaN,150.0,discover,142.0,...,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M,1.0
1,2987001,0,86401,29.00,W,2755,404.0,150.0,mastercard,102.0,...,32.0,1334x750,match_status:1,T,F,F,T,mobile,iOS Device,1.0
2,2987002,0,86469,59.00,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,F,F,T,T,desktop,Windows,1.0
3,2987003,0,86499,50.00,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,F,F,T,T,desktop,NaN,1.0
4,2987004,0,86506,50.00,H,4497,514.0,150.0,mastercard,102.0,...,24.0,1280x800,match_status:2,T,F,T,T,desktop,MacOS,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590535,3577535,0,15811047,49.00,W,6550,NaN,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590536,3577536,0,15811049,39.50,W,10444,225.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590537,3577537,0,15811079,30.95,W,12037,595.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590538,3577538,0,15811088,117.00,W,7826,481.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## IQR method

In [25]:
index,para = ot.outlier_detect_IQR(data=X,col='TransactionAmt',threshold=3)
print('Upper bound:',para[0],'\nLower bound:',para[1])

Num of outlier detected: 36422
Proportion of outlier detected 0.061675754394283196
Upper bound: 370.037 
Lower bound: -201.71599999999995


In [26]:
# check the 36422 found outliers
X.loc[index,'TransactionAmt'].sort_values()

242096      370.150
282969      370.150
169310      370.170
54938       370.170
64662       370.190
            ...    
384603     5543.230
248413     6085.230
296021     6450.970
274339    31937.391
274336    31937.391
Name: TransactionAmt, Length: 36422, dtype: float64

## Mean and Standard Deviation Method
outlier detection by Mean and Standard Deviation Method.

In [27]:
index,para = ot.outlier_detect_mean_std(data=X,col='TransactionAmt',threshold=3)
print('Upper bound:',para[0],'\nLower bound:',para[1])

Num of outlier detected: 10093
Proportion of outlier detected 0.01709113692552579
Upper bound: 852.5147424139221 
Lower bound: -582.4603896680457


In [28]:
# check the 10093 found outliers
X.loc[index,'TransactionAmt'].sort_values()

148925      852.770
272226      852.920
181902      852.920
280630      852.940
466209      852.950
            ...    
384603     5543.230
248413     6085.230
296021     6450.970
274339    31937.391
274336    31937.391
Name: TransactionAmt, Length: 10093, dtype: float64

## MAD method
outlier detection by Median and Median Absolute Deviation Method (MAD)

In [29]:
# too aggressive for our dataset, about 18% of cases are detected as outliers.
index = ot.outlier_detect_MAD(data=X,col='TransactionAmt',threshold=3.5)

Num of outlier detected: 59443
Proportion of outlier detected 0.10065871913841569


In [30]:
X.loc[index,'TransactionAmt'].sort_values()

324567      270.140
279689      270.160
56430       270.160
3449        270.240
468565      270.280
            ...    
384603     5543.230
248413     6085.230
296021     6450.970
274339    31937.391
274336    31937.391
Name: TransactionAmt, Length: 59443, dtype: float64

##  Imputation with arbitrary value
impute outliers with arbitrary value

In [37]:
# use any of the detection method above
index,para = ot.outlier_detect_arbitrary(data=X,col='TransactionAmt',upper_fence=40000,lower_fence=30)
print('Upper bound:',para[0],'\nLower bound:',para[1])

Num of outlier detected: 80631
Proportion of outlier detected 0.13653774511464084
Upper bound: 40000 
Lower bound: 30


In [38]:
X[300:325]

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo,had_id
300,2987300,0,91173,48.638,C,2115,415.0,185.0,mastercard,195.0,...,24.0,1366x768,match_status:2,T,F,T,T,desktop,Windows,1.0
301,2987301,0,91188,20.905,C,9633,296.0,185.0,visa,138.0,...,NaN,NaN,NaN,F,F,T,T,desktop,Windows,1.0
302,2987302,0,91215,48.638,C,2115,415.0,185.0,mastercard,195.0,...,24.0,1280x1024,match_status:2,T,F,T,T,desktop,Windows,1.0
303,2987303,0,91220,1351.550,W,11106,100.0,150.0,visa,226.0,...,NaN,NaN,NaN,F,F,T,T,mobile,Blade V6 Plus Build/MRA58K,1.0
304,2987304,0,91244,554.000,W,8528,215.0,150.0,visa,226.0,...,NaN,NaN,NaN,F,F,T,T,mobile,Blade V6 Plus Build/MRA58K,1.0
305,2987305,0,91257,48.638,C,2115,415.0,185.0,mastercard,195.0,...,24.0,1366x768,match_status:2,T,F,T,T,desktop,Windows,1.0
306,2987306,0,91323,20.000,W,12695,490.0,150.0,visa,226.0,...,NaN,NaN,NaN,F,F,T,T,mobile,SM-J700M Build/MMB29K,1.0
307,2987307,0,91330,159.950,W,12581,453.0,150.0,visa,226.0,...,32.0,960x540,match_status:2,T,F,T,T,mobile,SM-G530T,1.0
308,2987308,0,91333,25.950,W,5033,269.0,150.0,mastercard,224.0,...,32.0,1280x720,match_status:2,T,F,T,T,mobile,LGMP260 Build/NRD90U,1.0
309,2987309,0,91339,25.950,W,1974,111.0,150.0,visa,226.0,...,32.0,1334x750,match_status:1,T,F,T,T,mobile,iOS Device,1.0


In [39]:
# see index 306,308... have been replaced
data2 = ot.impute_outlier_with_arbitrary(data=X,outlier_index=index,
                                         value=-999,col=['TransactionAmt'])
data2[300:325]

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo,had_id
300,2987300,0,91173,48.638,C,2115,415.0,185.0,mastercard,195.0,...,24.0,1366x768,match_status:2,T,F,T,T,desktop,Windows,1.0
301,2987301,0,91188,-999.000,C,9633,296.0,185.0,visa,138.0,...,NaN,NaN,NaN,F,F,T,T,desktop,Windows,1.0
302,2987302,0,91215,48.638,C,2115,415.0,185.0,mastercard,195.0,...,24.0,1280x1024,match_status:2,T,F,T,T,desktop,Windows,1.0
303,2987303,0,91220,1351.550,W,11106,100.0,150.0,visa,226.0,...,NaN,NaN,NaN,F,F,T,T,mobile,Blade V6 Plus Build/MRA58K,1.0
304,2987304,0,91244,554.000,W,8528,215.0,150.0,visa,226.0,...,NaN,NaN,NaN,F,F,T,T,mobile,Blade V6 Plus Build/MRA58K,1.0
305,2987305,0,91257,48.638,C,2115,415.0,185.0,mastercard,195.0,...,24.0,1366x768,match_status:2,T,F,T,T,desktop,Windows,1.0
306,2987306,0,91323,-999.000,W,12695,490.0,150.0,visa,226.0,...,NaN,NaN,NaN,F,F,T,T,mobile,SM-J700M Build/MMB29K,1.0
307,2987307,0,91330,159.950,W,12581,453.0,150.0,visa,226.0,...,32.0,960x540,match_status:2,T,F,T,T,mobile,SM-G530T,1.0
308,2987308,0,91333,-999.000,W,5033,269.0,150.0,mastercard,224.0,...,32.0,1280x720,match_status:2,T,F,T,T,mobile,LGMP260 Build/NRD90U,1.0
309,2987309,0,91339,-999.000,W,1974,111.0,150.0,visa,226.0,...,32.0,1334x750,match_status:1,T,F,T,T,mobile,iOS Device,1.0


## Windsorization
top-coding & bottom coding (capping the maximum of a distribution at an arbitrarily set value,vice versa)


In [42]:
# use any of the detection method above
index,para = ot.outlier_detect_arbitrary(X,'TransactionAmt',40000,30)
print('Upper bound:',para[0],'\nLower bound:',para[1])

Num of outlier detected: 80631
Proportion of outlier detected 0.13653774511464084
Upper bound: 40000 
Lower bound: 30


In [43]:
para

(40000, 30)

In [44]:
# see index 306,308... have been replaced with top/bottom coding

data3 = ot.windsorization(data=X,col='TransactionAmt',para=para,strategy='both')
data3[300:325]

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo,had_id
300,2987300,0,91173,48.638,C,2115,415.0,185.0,mastercard,195.0,...,24.0,1366x768,match_status:2,T,F,T,T,desktop,Windows,1.0
301,2987301,0,91188,30.000,C,9633,296.0,185.0,visa,138.0,...,NaN,NaN,NaN,F,F,T,T,desktop,Windows,1.0
302,2987302,0,91215,48.638,C,2115,415.0,185.0,mastercard,195.0,...,24.0,1280x1024,match_status:2,T,F,T,T,desktop,Windows,1.0
303,2987303,0,91220,1351.550,W,11106,100.0,150.0,visa,226.0,...,NaN,NaN,NaN,F,F,T,T,mobile,Blade V6 Plus Build/MRA58K,1.0
304,2987304,0,91244,554.000,W,8528,215.0,150.0,visa,226.0,...,NaN,NaN,NaN,F,F,T,T,mobile,Blade V6 Plus Build/MRA58K,1.0
305,2987305,0,91257,48.638,C,2115,415.0,185.0,mastercard,195.0,...,24.0,1366x768,match_status:2,T,F,T,T,desktop,Windows,1.0
306,2987306,0,91323,30.000,W,12695,490.0,150.0,visa,226.0,...,NaN,NaN,NaN,F,F,T,T,mobile,SM-J700M Build/MMB29K,1.0
307,2987307,0,91330,159.950,W,12581,453.0,150.0,visa,226.0,...,32.0,960x540,match_status:2,T,F,T,T,mobile,SM-G530T,1.0
308,2987308,0,91333,30.000,W,5033,269.0,150.0,mastercard,224.0,...,32.0,1280x720,match_status:2,T,F,T,T,mobile,LGMP260 Build/NRD90U,1.0
309,2987309,0,91339,30.000,W,1974,111.0,150.0,visa,226.0,...,32.0,1334x750,match_status:1,T,F,T,T,mobile,iOS Device,1.0


## Discard outliers
Drop the cases that are outliers

In [45]:
# use any of the detection method above
index,para = ot.outlier_detect_arbitrary(X,'TransactionAmt',40000,30)
print('Upper bound:',para[0],'\nLower bound:',para[1])

Num of outlier detected: 80631
Proportion of outlier detected 0.13653774511464084
Upper bound: 40000 
Lower bound: 30


In [47]:
# drop the outlier.
# we can see no more observations have value >300 or <5. They've been removed.
data4 = ot.drop_outlier(data=X,outlier_index=index)
print(data4.TransactionAmt.max())
print(data4.TransactionAmt.min())

31937.391
30.0


## Mean/Median/Mode Imputation
replacing the outlier by mean/median/most frequent values of that variable

In [48]:
# use any of the detection method above
index,para = ot.outlier_detect_arbitrary(X,'TransactionAmt',40000,30)
print('Upper bound:',para[0],'\nLower bound:',para[1])
    

Num of outlier detected: 80631
Proportion of outlier detected 0.13653774511464084
Upper bound: 40000 
Lower bound: 30


In [49]:
# see index 258,263,271 have been replaced with mean

data5 = ot.impute_outlier_with_avg(data=X,col='TransactionAmt',
                                   outlier_index=index,strategy='mean')#median,mode
data5[300:325]

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo,had_id
300,2987300,0,91173,48.638000,C,2115,415.0,185.0,mastercard,195.0,...,24.0,1366x768,match_status:2,T,F,T,T,desktop,Windows,1.0
301,2987301,0,91188,135.027176,C,9633,296.0,185.0,visa,138.0,...,NaN,NaN,NaN,F,F,T,T,desktop,Windows,1.0
302,2987302,0,91215,48.638000,C,2115,415.0,185.0,mastercard,195.0,...,24.0,1280x1024,match_status:2,T,F,T,T,desktop,Windows,1.0
303,2987303,0,91220,1351.550000,W,11106,100.0,150.0,visa,226.0,...,NaN,NaN,NaN,F,F,T,T,mobile,Blade V6 Plus Build/MRA58K,1.0
304,2987304,0,91244,554.000000,W,8528,215.0,150.0,visa,226.0,...,NaN,NaN,NaN,F,F,T,T,mobile,Blade V6 Plus Build/MRA58K,1.0
305,2987305,0,91257,48.638000,C,2115,415.0,185.0,mastercard,195.0,...,24.0,1366x768,match_status:2,T,F,T,T,desktop,Windows,1.0
306,2987306,0,91323,135.027176,W,12695,490.0,150.0,visa,226.0,...,NaN,NaN,NaN,F,F,T,T,mobile,SM-J700M Build/MMB29K,1.0
307,2987307,0,91330,159.950000,W,12581,453.0,150.0,visa,226.0,...,32.0,960x540,match_status:2,T,F,T,T,mobile,SM-G530T,1.0
308,2987308,0,91333,135.027176,W,5033,269.0,150.0,mastercard,224.0,...,32.0,1280x720,match_status:2,T,F,T,T,mobile,LGMP260 Build/NRD90U,1.0
309,2987309,0,91339,135.027176,W,1974,111.0,150.0,visa,226.0,...,32.0,1334x750,match_status:1,T,F,T,T,mobile,iOS Device,1.0
